In [ ]:
import pickle
import torch
import sys
import os
from google.colab import drive

drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/OptimizedBERT')
original_sys_path = sys.path.copy()
sys.path.append('/content/drive/MyDrive/OptimizedBERT')

from sentence_evaluator import ModelSentenceEvaluator
from dataset_evaluator import DatasetEvaluator

sys.path.append('/content/drive/MyDrive/OptimizedBERT')

In [ ]:
file_path = "training_data.txt"
training_data = []
with open(file_path, 'r') as file:
    for line in file:
        training_data.append(line.strip())

training_data = list(set(training_data))
print("Training Dataset Length: ",len(training_data))

In [ ]:
# Load relevant models

def load_model(model_path):
    with open(model_path, 'rb') as f:
        return pickle.load(f)

folder = "models/"
ground_truth = ("original", load_model(folder + "original_model.pkl"))
models = {
    "half_precision": load_model(folder + "half_precision_model.pkl"),
    "6bit": load_model(folder + "6bit_model.pkl"),
    "8bit": load_model(folder + "8bit_model.pkl"),
    "16bit": load_model(folder + "16bit_model.pkl"),
    "gelu_approx": load_model(folder + "gelu_approx_model.pkl"),
    "layernorm_approx": load_model(folder + "layernorm_approx_model.pkl"),
    "softmax_approx": load_model(folder + "softmax_approx_model.pkl"),
    "all_approx": load_model(folder + "all_approx_model.pkl")
}

In [ ]:
# Analytics Setup
k=1000
desired_percentage=95
token_step=200
max_tokens=10000

In [ ]:
# Sentence Level Performance

evaluator = ModelSentenceEvaluator(models, ground_truth)
sentence = "The <mask> of the situation remained elusive despite exhaustive investigation."
evaluator.all_model_inference(sentence)
evaluator.plot_all_metrics(k, desired_percentage, token_step, max_tokens)

In [ ]:
# Dataset Level Performance

dataset_evaluator = DatasetEvaluator(models, ground_truth, training_data)
dataset_evaluator.evaluate(k,desired_percentage)
avg_metrics = dataset_evaluator.average_metrics()
dataset_evaluator.plot_metrics()